In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats.mstats import winsorize
import numpy as np
from scipy.stats import norm
from scipy import stats
from statsmodels.formula.api import ols
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')

In [3]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [38]:
df = pd.read_csv('./dataset/윈저데이터셋.csv', encoding='CP949')
df = df[['[제조]매출액증가율',
       '[제조]유형자산증가율', '[제조]비유동자산증가율', '[제조]총자본증가율', '[제조]자기자본증가율',
       '[제조]유동자산증가율', '[제조]순이익증가율', '[제조]재고자산증가율', '[제조]매출액총이익률',
       '[제조]매출액순이익률', '[제조]총자본사업이익률', '[제조]총자본순이익률', '[제조]자기자본순이익률',
       '[제조]1주당순이익', '[제조]1주당매출액', '[제조]총자본회전률', '[제조]자기자본회전률', '[제조]비유동자산회전률',
       '[제조]재고자산회전률', '[제조]매출채권회전률', '[제조]매입채무회전률', '[제조]매입채무회전기간',
       '[제조]재고자산회전기간', '[제조]매출채권회전기간', '[제조]1회전기간', '[제조]운전자본회전률',
       '[제조]순운전자본회전률', '[제조]유동자산회전률', '[제조]당좌자산회전률', '[제조]유동비율', '[제조]당좌비율',
       '[제조]현금비율', '[제조]부채비율', '[제조]순운전자본비율', '[제조]자기자본배율', '[제조]비유동비율',
       '[제조]유동자산구성비율', '[제조]자기자본구성비율', '[제조]부가가치', '[제조]총자본투자효율', '[제조]설비투자효율',
       '[제조]기계투자효율', '[제조]부가가치율', '[제조]노동소득분배율', '[제조]자본분배율', '[제조]이윤분배율',
       '[제조]유형자산회전율', '[제조]이자보상배율(이자비용)','label']]

In [39]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [40]:
X_samp_1, y_samp_1 = RandomUnderSampler(random_state=0,sampling_strategy=0.1).fit_resample(X, y)

print('RS 적용 전 학습용 피처/레이블 데이터 세트: ', X.shape, y.shape)
print('RS 적용 후 학습용 피처/레이블 데이터 세트: ', X_samp_1.shape, y_samp_1.shape)
print('RS 적용 후 레이블 값 분포: \n', pd.Series(y_samp_1).value_counts())

RS 적용 전 학습용 피처/레이블 데이터 세트:  (10652, 48) (10652,)
RS 적용 후 학습용 피처/레이블 데이터 세트:  (1793, 48) (1793,)
RS 적용 후 레이블 값 분포: 
 0    1630
1     163
Name: label, dtype: int64


In [41]:
Under_samp = pd.concat([X_samp_1,y_samp_1], axis=1)
Under_samp

,[제조]매출액증가율,[제조]유형자산증가율,[제조]비유동자산증가율,[제조]총자본증가율,[제조]자기자본증가율,[제조]유동자산증가율,[제조]순이익증가율,[제조]재고자산증가율,[제조]매출액총이익률,[제조]매출액순이익률,...,[제조]총자본투자효율,[제조]설비투자효율,[제조]기계투자효율,[제조]부가가치율,[제조]노동소득분배율,[제조]자본분배율,[제조]이윤분배율,[제조]유형자산회전율,[제조]이자보상배율(이자비용),label
0,70.32,24.04,13.16,11.44,14.86,8.56,70.56,-0.29,63.90,12.48,...,19.61,45.95,277.98,41.21,32.48,67.52,30.29,1.20,5.33,0
1,3.73,19.78,11.25,10.50,7.14,9.87,-70.45,5.95,16.58,2.77,...,21.78,101.69,202.22,23.14,39.84,60.16,11.99,4.46,6.86,0
2,-40.28,64.14,25.38,25.79,-69.15,26.01,0.00,0.00,0.00,-157.46,...,-10.53,-209.51,-529.47,-79.70,0.00,0.00,0.00,1.03,-4.52,0
3,9.32,-4.75,17.43,14.64,16.92,12.52,72.84,13.96,52.94,14.29,...,28.48,122.05,521.57,41.57,52.73,47.27,34.39,2.85,134.14,0
4,75.57,5.79,30.74,7.85,6.57,0.91,0.00,7.10,40.49,10.40,...,18.69,179.86,41241.44,42.20,70.83,29.17,24.65,4.38,3145.21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,-58.60,49.06,45.58,-46.18,-92.81,-67.91,0.00,-52.79,6.28,-62.75,...,-10.53,-60.99,-2412.95,-40.77,0.00,0.00,0.00,1.79,-6.63,1
1789,-12.68,-4.08,-9.92,15.93,-14.52,31.14,19.55,58.91,26.96,-40.47,...,2.13,10.53,112.68,4.80,184.40,-84.40,-181.31,2.15,-11.10,1
1790,-10.85,-12.11,-26.60,2.10,6.24,5.00,0.00,11.81,23.44,6.97,...,12.86,368.59,21357.30,15.40,18.52,81.48,45.26,22.39,12.17,1
1791,-4.88,-33.36,-34.42,-39.94,-36.03,-42.77,-206.32,-17.81,9.37,-27.57,...,-10.53,-114.84,-161.78,-24.26,0.00,0.00,0.00,3.79,-15.97,1


In [35]:
Under_samp.to_csv('./dataset/Undersampling.csv',encoding='CP949', index=False)

In [32]:
y_samp_1

0       0
1       0
2       0
3       0
4       0
       ..
1788    1
1789    1
1790    1
1791    1
1792    1
Name: label, Length: 1793, dtype: int64

In [26]:
X_samp_1

,[제조]매출액증가율,[제조]유형자산증가율,[제조]비유동자산증가율,[제조]총자본증가율,[제조]자기자본증가율,[제조]유동자산증가율,[제조]순이익증가율,[제조]재고자산증가율,[제조]매출액총이익률,[제조]매출액순이익률,...,[제조]부가가치,[제조]총자본투자효율,[제조]설비투자효율,[제조]기계투자효율,[제조]부가가치율,[제조]노동소득분배율,[제조]자본분배율,[제조]이윤분배율,[제조]유형자산회전율,[제조]이자보상배율(이자비용)
0,70.32,24.04,13.16,11.44,14.86,8.56,70.56,-0.29,63.90,12.48,...,128478.83,19.61,45.95,277.98,41.21,32.48,67.52,30.29,1.20,5.33
1,3.73,19.78,11.25,10.50,7.14,9.87,-70.45,5.95,16.58,2.77,...,77022.04,21.78,101.69,202.22,23.14,39.84,60.16,11.99,4.46,6.86
2,-40.28,64.14,25.38,25.79,-69.15,26.01,0.00,0.00,0.00,-157.46,...,-14075.53,-10.53,-209.51,-529.47,-79.70,0.00,0.00,0.00,1.03,-4.52
3,9.32,-4.75,17.43,14.64,16.92,12.52,72.84,13.96,52.94,14.29,...,66967.53,28.48,122.05,521.57,41.57,52.73,47.27,34.39,2.85,134.14
4,75.57,5.79,30.74,7.85,6.57,0.91,0.00,7.10,40.49,10.40,...,5369.64,18.69,179.86,41241.44,42.20,70.83,29.17,24.65,4.38,3145.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,-58.60,49.06,45.58,-46.18,-92.81,-67.91,0.00,-52.79,6.28,-62.75,...,-15435.00,-10.53,-60.99,-2412.95,-40.77,0.00,0.00,0.00,1.79,-6.63
1789,-12.68,-4.08,-9.92,15.93,-14.52,31.14,19.55,58.91,26.96,-40.47,...,2002.69,2.13,10.53,112.68,4.80,184.40,-84.40,-181.31,2.15,-11.10
1790,-10.85,-12.11,-26.60,2.10,6.24,5.00,0.00,11.81,23.44,6.97,...,69938.32,12.86,368.59,21357.30,15.40,18.52,81.48,45.26,22.39,12.17
1791,-4.88,-33.36,-34.42,-39.94,-36.03,-42.77,-206.32,-17.81,9.37,-27.57,...,-15435.00,-10.53,-114.84,-161.78,-24.26,0.00,0.00,0.00,3.79,-15.97


In [37]:
x_train, x_test, y_train, y_test = train_test_split(X_samp_1, y_samp_1, test_size=0.2, random_state=0)

In [36]:
X_samp_2, y_samp_2 = RandomOverSampler(random_state=0,sampling_strategy=0.25).fit_resample(X_samp_1, y_samp_1)

print('RS 적용 전 학습용 피처/레이블 데이터 세트: ', X_samp_1.shape, y_samp_1.shape)
print('RS 적용 후 학습용 피처/레이블 데이터 세트: ', X_samp_2.shape, y_samp_2.shape)
print('RS 적용 후 레이블 값 분포: \n', pd.Series(y_samp_2).value_counts())

RS 적용 전 학습용 피처/레이블 데이터 세트:  (1793, 48) (1793,)
RS 적용 후 학습용 피처/레이블 데이터 세트:  (2037, 48) (2037,)
RS 적용 후 레이블 값 분포: 
 0    1630
1     407
Name: label, dtype: int64
